# Elasticsearh - usign python API

### Import it and created client object

In [ ]:
from elasticsearch import Elasticsearch, helpers, exceptions as es_exceptions
from elasticsearch.helpers import bulk,scan

es = Elasticsearch(['atlas-kibana.mwt2.org:9200'],timeout=60)

### Check we can connect to the cluster

In [ ]:
# define our address so we dont have to type it all the time
es.cluster.health()

### Create a bunch of documents

In [ ]:
import random
allEvents=[]
for e in range(1000):
    E=random.gauss(1000., 30.)
    pT=random.gauss(100., 50.)
    allEvents.append({'_type':'event', 'eventnr':e, 'E':E, 'pT':pT})

### Index documents one by one

takes around 70 seconds.

In [ ]:
from time import time
start=time()
for nr, event in enumerate(allEvents):
    es.create(index='my_events', id=nr, body=event)
print("it took", time()-start,"seconds.")

### Bulk indexing of same events

In [ ]:
# clean all
try:
    es.indices.delete(index='my_events')
except:
    print("not there?")
    
start=time()
try:
   res = helpers.bulk(es, allEvents, index='my_events', raise_on_exception=True,request_timeout=60)
except es_exceptions.ConnectionError as e:
   print('ConnectionError ', e)
except es_exceptions.TransportError as e:
   print('TransportError ', e)
except helpers.BulkIndexError as e:
   print(e[0])
   for i in e[1]:
      print(i)
except Exception as e:
   print('Something seriously wrong happened.',e)

print("it took", time()-start,"seconds.")

### Search for a document

In [ ]:
my_query={
    "size": 15,
    "query":{
       "bool":{
            "must":[
                {'range': {'pT': {'gte': 100, 'lt': 120}}},
                {"range" : { "E" :{'gte': 200 }}}
            ]
        }
    }
}

res = es.search(index='my_events', body=my_query )
for r in res['hits']['hits']:
    print(r)

### Perform aggregated search

In [ ]:
my_query={
    "aggs" : {
        "pt_bins" : {
            "histogram" : {
                "field" : "pT",
                "interval" : 50
            }
        }
    }
}

res = es.search(index='my_events', body=my_query )
print(res['aggregations'])

### Clean up

In [ ]:
es.indices.delete(index='my_events')